# Itinerary Planning Problem
## As an Extension of Distance Constrained Vehicle Routing Problem Formulation

<u>Formulated By:</u><br>
Chan Shao Jing<br>
Jain Karisha<br>
Low Jin Teng Jackson <br>
Yeo Galvin <br>

### 1. Load Packages

In [1]:
import pandas as pd
import numpy as np
import folium
import mpu
from folium.plugins import BeautifyIcon
import sys
import math
import requests
import datetime
import time
import json
import requests
from geopy.geocoders import Nominatim
import openrouteservice as ors

from rsome import ro
from rsome import grb_solver as grb
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.display import clear_output

### 2. Load Datasets

In [2]:
# Loading datasets
activities_df = pd.read_csv('activities.csv')
accomodations_df = pd.read_csv("accomodations.csv")

# Assumptions for accomodations dataset
# 1. Cost is per night hotel price for a standard room of 2
# 2. Booked 3 months in advance (no early/late charges)

accomodations_df[["estduration", "mustdoflag"]] = 0, 0
data = pd.concat([accomodations_df, activities_df], ignore_index = True)
data

,name,address,subtype,cost,inoutflag,estduration,mustdoflag
0,Marina Bay Sands,"10 Bayfront Ave, Singapore 018956",hotel,799.00,indoor,0.0,0
1,Park Avenue Changi Hotel,"2 Changi Business Park Ave 1, Singapore 486015",hotel,164.05,indoor,0.0,0
2,Hard Rock Hotel,"8 Sentosa Gateway, 098269",hotel,580.00,indoor,0.0,0
3,Shangri La Hotel,"22 Orange Grove Rd, Singapore 258350",hotel,304.00,indoor,0.0,0
4,Genting Hotel Jurong,"2 Town Hall Link, Singapore 608516",hotel,192.00,indoor,0.0,0
5,Ibis Styles on MacPherson,"401 MacPherson Rd, Singapore 368125",hotel,116.00,indoor,0.0,0
6,Universal Studios Singapore,"8 Sentosa Gateway, 098269",theme park,61.00,outdoor,8.0,1
7,Singapore Art Museum,"71 Bras Basah Rd, Singapore 189555",museum,10.00,indoor,3.0,1
8,Gardens By The Bay,"18 Marina Gardens Dr, Singapore 018953",sightseeing,0.00,outdoor,4.0,1
9,Singapore Zoo,"80 Mandai Lake Rd, 729826",theme park,48.00,outdoor,8.0,1


### 3. Generating Coordinates using Geopy Nominatim Package

In [3]:
# Generating and parsing latitude and longitude of every location
# Estimated to take around a minute
geolocator = Nominatim(user_agent='prescriptiveproj')

def coordinate(x):
    try:
        coord = [geolocator.geocode(x).latitude, geolocator.geocode(x).longitude]
    except:
        coord = None
    return coord
    
data['coordinates'] = data['address'].apply(lambda x: coordinate(x))
data[['lat','long']] = pd.DataFrame(data.coordinates.tolist(), index = data.index)
data

,name,address,subtype,cost,inoutflag,estduration,mustdoflag,coordinates,lat,long
0,Marina Bay Sands,"10 Bayfront Ave, Singapore 018956",hotel,799.00,indoor,0.0,0,"[1.2836965, 103.8607226]",1.283697,103.860723
1,Park Avenue Changi Hotel,"2 Changi Business Park Ave 1, Singapore 486015",hotel,164.05,indoor,0.0,0,"[1.3358477, 103.9630856]",1.335848,103.963086
2,Hard Rock Hotel,"8 Sentosa Gateway, 098269",hotel,580.00,indoor,0.0,0,"[1.25574585, 103.82123439403357]",1.255746,103.821234
3,Shangri La Hotel,"22 Orange Grove Rd, Singapore 258350",hotel,304.00,indoor,0.0,0,"[1.3112474, 103.8267568]",1.311247,103.826757
4,Genting Hotel Jurong,"2 Town Hall Link, Singapore 608516",hotel,192.00,indoor,0.0,0,"[1.3315616, 103.7392596]",1.331562,103.739260
5,Ibis Styles on MacPherson,"401 MacPherson Rd, Singapore 368125",hotel,116.00,indoor,0.0,0,"[1.3317706, 103.8792945]",1.331771,103.879295
6,Universal Studios Singapore,"8 Sentosa Gateway, 098269",theme park,61.00,outdoor,8.0,1,"[1.25574585, 103.82123439403357]",1.255746,103.821234
7,Singapore Art Museum,"71 Bras Basah Rd, Singapore 189555",museum,10.00,indoor,3.0,1,"[1.2977042, 103.8507745]",1.297704,103.850774
8,Gardens By The Bay,"18 Marina Gardens Dr, Singapore 018953",sightseeing,0.00,outdoor,4.0,1,"[1.2848639, 103.8640308]",1.284864,103.864031
9,Singapore Zoo,"80 Mandai Lake Rd, 729826",theme park,48.00,outdoor,8.0,1,"[1.4037076, 103.79403742902613]",1.403708,103.794037


In [4]:
# The geolocator generated coordinates for Singapore Zoo do not have any road routes near it 
# Which will cause our openrouteservice API to fail later on
# Thus, we will need to manually insert the coordinates of the dropoff point for Singapore Zoo 
data.loc[9, 'coordinates'] = "[1.404651, 103.788059]"
data.loc[9, 'lat'] = "1.404651"
data.loc[9, 'long'] = "103.788059"
data

,name,address,subtype,cost,inoutflag,estduration,mustdoflag,coordinates,lat,long
0,Marina Bay Sands,"10 Bayfront Ave, Singapore 018956",hotel,799.00,indoor,0.0,0,"[1.2836965, 103.8607226]",1.283697,103.860723
1,Park Avenue Changi Hotel,"2 Changi Business Park Ave 1, Singapore 486015",hotel,164.05,indoor,0.0,0,"[1.3358477, 103.9630856]",1.335848,103.963086
2,Hard Rock Hotel,"8 Sentosa Gateway, 098269",hotel,580.00,indoor,0.0,0,"[1.25574585, 103.82123439403357]",1.255746,103.821234
3,Shangri La Hotel,"22 Orange Grove Rd, Singapore 258350",hotel,304.00,indoor,0.0,0,"[1.3112474, 103.8267568]",1.311247,103.826757
4,Genting Hotel Jurong,"2 Town Hall Link, Singapore 608516",hotel,192.00,indoor,0.0,0,"[1.3315616, 103.7392596]",1.331562,103.73926
5,Ibis Styles on MacPherson,"401 MacPherson Rd, Singapore 368125",hotel,116.00,indoor,0.0,0,"[1.3317706, 103.8792945]",1.331771,103.879295
6,Universal Studios Singapore,"8 Sentosa Gateway, 098269",theme park,61.00,outdoor,8.0,1,"[1.25574585, 103.82123439403357]",1.255746,103.821234
7,Singapore Art Museum,"71 Bras Basah Rd, Singapore 189555",museum,10.00,indoor,3.0,1,"[1.2977042, 103.8507745]",1.297704,103.850774
8,Gardens By The Bay,"18 Marina Gardens Dr, Singapore 018953",sightseeing,0.00,outdoor,4.0,1,"[1.2848639, 103.8640308]",1.284864,103.864031
9,Singapore Zoo,"80 Mandai Lake Rd, 729826",theme park,48.00,outdoor,8.0,1,"[1.404651, 103.788059]",1.404651,103.788059


### 4. Variables Initialization

In [5]:
# User inputs for start and end time (in 24-hour format)
starttime = 9 
endtime = 22

# Convert to datetime format
starttimeofday = datetime.timedelta(hours = starttime)
endtimeofday = datetime.timedelta(hours = endtime) 

# User inputs for start and end date
startdate = datetime.date(2023, 4, 20) 
enddate = datetime.date(2023, 4, 24)

daysdiff = enddate - startdate # Inclusive of End Date
days = daysdiff.days + 1
hoursperday = (endtimeofday-starttimeofday).total_seconds() / 3600
print("Hours per day: ", hoursperday)

Hours per day:  13.0


In [6]:
# Assigning named variables to data for clearer programming
lon = data['long']
lat = data['lat']
name = data['name']
activity_type = data['subtype']
inoutflag = data['inoutflag']
city = "Singapore"
estduration = data['estduration']
mustdoflag = data['mustdoflag']
cost = data['cost']

N = len(data)
accomodations_count = len(accomodations_df)
activities_count = len(activities_df)

# Preset time limit to generate solution
timelimit = 120

In [7]:
# Distance and travelling time calculation
lonlat = [[lon[i], lat[i]] for i in range(N)]
body = {"locations":lonlat[0:50], "metrics":["distance"]} ## maximum 50 locations

# Calling openrouteservice API to get distances from each location to every other location
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf624835774ebd2fcc4d7baf8b08f85b315072',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)
print(call.status_code, call.reason)
data_req = call.json()

# Estimated average speed of a car in Singapore according to https://www.numbeo.com/traffic/country_result.jsp?country=Singapore
avg_speed_car = 18.96 / (31.82 / 60) # km/h
distance_mat = (np.array(data_req['distances']) / 1000)
travellingtime_mat = distance_mat / avg_speed_car

# To add estimated travelling time required to estimated duration of activity j
travelact_mat = np.zeros((N,N))
for i in range(len(travellingtime_mat)):
    for j in range(len(travellingtime_mat[i])):
        travelact_mat[i,j] = travellingtime_mat[i,j] + estduration[j]

200 OK


### 5. Plot Points on Map

In [8]:
SGmap = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11) #Stamen Terrain, OpenStreetMap
for i in range(N):
    if i < accomodations_count:
            folium.Marker(
        location=[lat[i], lon[i]],
        icon=BeautifyIcon(
            icon_shape='marker',
            number=int(i + 1),
            spin=True,
            text_color='blue',
            background_color="#FFF",
            inner_icon_style="font-size:12px;padding-top:-5px;"
        )
    ).add_to(SGmap)
    else:
        folium.Marker(
            location=[lat[i], lon[i]],
            icon=BeautifyIcon(
                icon_shape='marker',
                number=int(i + 1),
                spin=True,
                text_color='red',
                background_color="#FFF",
                inner_icon_style="font-size:12px;padding-top:-5px;"
            )
        ).add_to(SGmap)
SGmap

### 6. Generating Weather Forecast using WeatherAPI

In [9]:
# Loading weather codes dataset derived from WeatherAPI website
# Each weather code is representative of a certain specific weather condition
# Derived column is inserted manually to generalise the weather conditions
weather = pd.read_csv('weather_conditions.csv')
api_key = "214f76ee879149de9b774958232503" 

# Putting the weather code to weather condition mapping into a dictionary
dictionary = {}
code = weather['code']
condition = weather['derived']
for i in range(len(code)):
    dictionary.update({code[i]:condition[i]})

In [10]:
# Make a request to the WeatherAPI.com API for the hourly weather data for Singapore on the specified date
def get_weather(date, api_key, time_of_day):
    url = f"http://api.weatherapi.com/v1/forecast.json?key={api_key}&q=Singapore&dt={date}&hour=0-23"
    response = requests.get(url)
    data = response.json()
    day_condition = data["forecast"]["forecastday"][0]["hour"][time_of_day]["condition"]['code']
    
    return day_condition

# Get weather forecast for a range of dates
def forecast_over_range(dictionary, start_date, end_date, start_time, end_time):
    current_date = start_date
    conditionlist = []
    daylist = []
    new_start = start_time
    while current_date <= end_date:
        while new_start <= end_time:
            data = get_weather(current_date, api_key, new_start)
            condition = dictionary.get(data)
            conditionlist.append(condition)
            new_start += 1     
        daylist.append(conditionlist)
        conditionlist = []
        new_start = start_time
        current_date += datetime.timedelta(days=1)
        
    return daylist

# Process data to make it human readable 
def process_forecast(dictionary, forecast, start_time, end_time):
    finallist = []
    hours = end_time - start_time
    for i in range(len(forecast)):
        list = []
        count = 0
        current = forecast[i][0]
        for j in range(1,hours+1):
            
            next = forecast[i][j]
            if (next == current):
                count += 1
                current = next
                if (j == hours):
                    list.append("From " + "{:02d}".format(start_time+j-1-(count)) + "00hrs to " + "{:02d}".format(start_time+j+1) + "00hrs: " + next)
                continue
            else:
                list.append("From " + "{:02d}".format(start_time+j-1-(count)) + "00hrs to " + "{:02d}".format(start_time+j) + "00hrs: " + current)
                if (j == hours):
                    list.append("From " + "{:02d}".format(start_time+j) + "00hrs to " + "{:02d}".format(start_time+j+1) + "00hrs: " + next)
                current = next
                count = 0
            
        finallist.append(list)
        
    return finallist

In [11]:
forecast = forecast_over_range(dictionary, startdate, enddate, starttime, endtime-1)
forecast_results = process_forecast(dictionary, forecast, starttime, endtime-1)
forecast_results

[['From 0900hrs to 1100hrs: Light Rain',
  'From 1100hrs to 1200hrs: Heavy Rain',
  'From 1200hrs to 1400hrs: Light Rain',
  'From 1400hrs to 1500hrs: Moderate Rain',
  'From 1400hrs to 2200hrs: No Rain'],
 ['From 0900hrs to 1000hrs: Light Rain',
  'From 1000hrs to 1100hrs: Heavy Rain',
  'From 1100hrs to 1500hrs: Light Rain',
  'From 1500hrs to 1600hrs: No Rain',
  'From 1600hrs to 1700hrs: Light Rain',
  'From 1700hrs to 1800hrs: No Rain',
  'From 1800hrs to 1900hrs: Light Rain',
  'From 1900hrs to 2000hrs: No Rain',
  'From 1900hrs to 2200hrs: Light Rain'],
 ['From 0900hrs to 1000hrs: Heavy Rain',
  'From 1000hrs to 1100hrs: Light Rain',
  'From 1100hrs to 1500hrs: Heavy Rain',
  'From 1500hrs to 1600hrs: Light Rain',
  'From 1600hrs to 1700hrs: Heavy Rain',
  'From 1700hrs to 2100hrs: Light Rain',
  'From 2100hrs to 2200hrs: Heavy Rain'],
 ['From 0900hrs to 1000hrs: Drizzle',
  'From 1000hrs to 1100hrs: Heavy Rain',
  'From 1100hrs to 1200hrs: Drizzle',
  'From 1200hrs to 1300hrs: 

### 7. Problem Formulation as a Function
Let  $\mathcal{A} = \{0, 1,2,\ldots, N\}$ where $N\$ = No. of Accomodations + No. of Activities

Let  $\mathcal{H} = \{0, 1,2,\ldots, M\}$ where $M$ = No. of Accomodations and $\mathcal{H} \subset \mathcal{A}$.

Let $u_i$ be a dummy variable for  $i=1,\ldots,N$  <br><br>
Take $t_{ij} > 0$ to be the sum of the time taken to travel from activity $i$ to activity $j$ and the estimated duration required to be spent at $j$ $\forall i \in \mathcal{A}, j \in \mathcal{A}$<br><br>
Take $R_i$ as a flag for activity $i$ $\forall i \in \mathcal{A}$, 1 if user specified that it must be done this trip, 0 otherwise <br><br>
Take $C_j$ as the cost for activity $j$ $\forall j \in \mathcal{A}$

Let $K$ be the number of days of the trip and also number of subtours in the network optimization problem

Let $D$ be the duration desired per day

Let $T$ be the number of activities required per day, which is the maximum number of activities given by the performance of the model

Let $B$ be the budget specified by the user of the trip, if none is specified the model will assign $1,000,000 by default which is a sufficiently high number that can cover all the cost of any trip



$$
x_{ij} = \begin{cases} 
1 & \text{the path goes from activity } i \text{ to activity } j, \quad  \forall i \in \mathcal{A}, j \in \mathcal{A}\\ 
0 & \text{otherwise} 
\end{cases}\\
h_{i} = \begin{cases} 
1 & \text{accomodation } i \text { has been selected}, \quad  \forall i \in \mathcal{H}\\ 
0 & \text{otherwise} 
\end{cases}\\
y_{i} = \begin{cases} 
1 & \text{activity } i \text { has been selected}, \quad  \forall i \in \mathcal{A}\\ 
0 & \text{otherwise} 
\end{cases}
$$




$$
\begin{align}
\min & \sum_{i\in \mathcal{A}}\sum_{j \in \mathcal{A}}t_{ij}x_{ij} \\
\mbox{s.t.} & \sum_{i \in \mathcal{A}}x_{ij}\leq1 \quad && \forall j \in \mathcal{A}\backslash \left \{ H \right \} \quad && (1)\\
&\sum_{i \in \mathcal{A}}x_{ji}\leq1 \quad && \forall i \in \mathcal{A}\backslash \left \{ H \right \} \quad && (2)\\
&\sum_{j \in \mathcal{A}}x_{ij} = Kh_{i}\quad && \forall i \in \mathcal{H} \quad && (3)\\
&\sum_{j \in \mathcal{A}}x_{ji} = Kh_{i}\quad && \forall i \in \mathcal{H} \quad && (4)\\
&\sum_{i \in \mathcal{H}}h_{i}=1&& \quad && (5)\\
&\sum_{i\in \mathcal{A}} x_{ij} = x_{ji} \quad && \forall j \in \mathcal{A}\backslash \left \{ H \right \} \quad && (6)\\
&u_j-u_i\geq t_{ij}-D(1-x_{ij}) && \forall i,j \in \mathcal{A}\backslash\{H\}, i\neq j, d_{ij} \leq D \quad && (7)\\
&0 \leq u_i \leq D && \forall i \in \mathcal{A}\backslash \{H\} \quad && (8)\\
&\sum_{i \in \mathcal{A}}y_{i} \geq T && \quad && (9)\\
&y_{i} \geq  R_{i} && \forall i \in \mathcal{A}\backslash \{H\}\quad && (10)\\
&y_{j} \leq  \sum_{i \in \mathcal{A}}x_{ij} && \forall j \in \mathcal{A}\quad && (11)\\
&\sum_{i \in \mathcal{A}}\sum_{j \in \mathcal{A}}x_{ij}C_{j} <= B && \quad && (12)\\
&x_{ij}\in \{0,1\} \quad && \forall i,j \in \mathcal{A} \quad && (13)\\
&h_{i}\in \{0,1\} \quad && \forall i \in \mathcal{H} \quad && (14)\\
&y_{i}\in \{0,1\} \quad && \forall i \in \mathcal{A} \quad && (15)\\
&x_{ii} = 0 \quad && \forall i \in \mathcal{A} \quad && (16)\\
\end{align}
$$

<u>Constraints:</u> <br>
(1 & 2) All activities except for accomodations occurs less than or equals once<br>
(3 & 4) To initiate variable $h_{i}$ and to specify number of subtours (or number of days of the trip)<br>
(5) Ensure only 1 accomodation is chosen<br>
(6) Flow conservation of all nodes except accomodation nodes (i.e. ensure inflow = outflow)<br>
(7 & 8) Extension of MTZ problem specific for the Distance Constrained Vehicle Routing Problem<br>
(9) To ensure that the number of activities is more than the number of activities possible for the trip. <br>
(10) To ensure that each must-do activity flagged by the user is chosen for the trip itinerary <br>
(11) To initiate variable y where if any x is 0 for the activity, its corresponding y will be 0 too <br>
(12) To ensure all activities are within the budget <br>
(13) $x_{ij}$ binary<br>
(14) $h_{i}$ binary<br>
(15) $y_{i}$ binary<br>
(16) To eliminate arcs from every activity to itself generated due to matrix generation in Python<br>

In [12]:
def itineraryplanner(N, accomodations_count, days, hoursperday, no_activities, timelimit = 120, budget = 100000):
    # Itinerary planner model
    m = ro.Model('Itinerary Planner')
    
    # Decision variables
    x = m.dvar( (N, N), 'B' )
    h = m.dvar( accomodations_count, 'B' )
    y = m.dvar( N, 'B' )
    u = m.dvar( N ) # Dummy variable
    
    # Objective function
    m.min( (travelact_mat * x).sum() )

    # Constraints 1 & 2: to ensure all activities (excluding accomodations) occurs less than or equals once
    m.st( x[:,j].sum() <= 1 for j in range(accomodations_count, N) ) 
    m.st( x[i,:].sum() <= 1 for i in range(accomodations_count, N) )

    # Constraints 3 & 4: initiate h binary variable if accomodation chosen =1 otherwise 0 
    # Each accomodation has an inflow and outflow exactly equals to the number of days
    for i in range(accomodations_count):
        m.st( h[i] * days == sum(x[i,j] for j in range(N)), h[i] * days == sum(x[j,i] for j in range(N)) )

    # Constraint 5: to ensure only 1 accomodation chosen
    m.st( sum(h[i] for i in range(accomodations_count)) == 1 )
    
    # Constraint 6: flow conservation for activities
    for j in range(accomodations_count, N):
        m.st( sum(x[i,j] for i in range(N)) == sum(x[j,i] for i in range(N)) )

    # Constraints 7 & 8: constraints to ensure each subtour has a constrained travelling time (extension of MTZ Model)
    for i in range(accomodations_count, N):
        m.st( u[i] <= hoursperday, u[i] >= 0 )
        for j in range(accomodations_count, N):
            if j != i and travelact_mat[i,j] <= hoursperday:
                m.st( u[j] - u[i] >= travelact_mat[i,j] - hoursperday * (1 - x[i,j]) )
                
    # Constraint 9: ensure total number of activities chosen more than or equals possible number of activities
    m.st( sum(y[i] for i in range(N)) >= no_activities )
    
    # Constraint 10: ensure flagged must do activities are chosen
    for i in range(accomodations_count, N):
        m.st( y[i] >= mustdoflag[i] )
    
    # Constraint 11: to initiate variable y where if any x is 0 for the activity, its corresponding y will be 0 too
    for j in range(N):
        m.st( y[j] <= sum(x[i,j] for i in range (N)) )
    
    # Constraint 12: to ensure all activities within budget
    m.st( sum(x[i,j] * cost[j] for i in range(N) for j in range(N)) <= budget )
    
    # Constraints 13, 14 & 15: binary constraints accounted for in declaring of decision variables as binary
    
    # Constraint 16: to eliminate arcs from every activity to itself
    m.st( x[i,i] == 0 for i in range(N) )   
    
    # Solver with time limit
    m.solve( grb, params = {'TimeLimit': timelimit} )
    
    return m.get(), x.get(), y.get(), m.solution.status

### 8. Run Model with Dynamic Activity Constraint

In [13]:
# Binary search to find the best number of activities within time limit specified
# Estimated to take around 5-10 minutes (results may defer depending on CPU)
trycount = 0
outputcount = 0
status = 0

low = days + 1 # Minimum number of activities (at least 1 activity per day) + 1 accommodation
high = activities_count + 1 # Maximum number of actvitiies + 1 accommodation
    
while low <= high:
    mid = (low+high) // 2 
    print('Number of Activities: ', mid - 1)
    outputcount += 1
    try:
        trycount += 1
        print('Attempt no.', trycount)
        results, x_star, y_star, status = itineraryplanner(N, accomodations_count, days, hoursperday, mid, timelimit)
        if (status == 2): # Optimal solution found
            low = mid + 1
            print('Solution Found!') # Save last optimal solution
            last_r = results 
            last_xstar = x_star
            last_ystar = y_star
            last_status = status
        elif (status == 9): # Time limit exceeded
            high = mid - 1
            print('Time Limit Reached!')
            if outputcount == 10:
                outputcount=0
                clear_output()
    except:
        print('Unable to Find Solution!') # Error/no solution found
        high = mid - 1
        if outputcount == 10:
            outputcount=0
            clear_output()
                
# Set results to last optimal solution found
results, x_star, y_star, status = last_r, last_xstar, last_ystar, last_status

Number of Activities:  15
Attempt no. 1
Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05
Being solved by Gurobi...
Solution status: 2
Running time: 0.4120s
Solution Found!
Number of Activities:  21
Attempt no. 2
Being solved by Gurobi...
Solution status: 2
Running time: 11.0790s
Solution Found!
Number of Activities:  24
Attempt no. 3
Being solved by Gurobi...
Solution status: 2
Running time: 96.8320s
Solution Found!
Number of Activities:  25
Attempt no. 4
Being solved by Gurobi...
Solution status: 9
Running time: 120.0810s
Unable to Find Solution!


In [14]:
print("Minimised distance: ", "{:.1f}".format(results), "km")

Minimised distance:  60.4 km


### 8.5. Saving and Loading Results (Optional)

In [15]:
#final_results = [results, x_star, y_star, status]
#string = 'Itinerary.pickle'
#with open(string, 'wb') as f:
#   pickle.dump(final_results, f)

In [16]:
#loaded_results = pickle.load( open( 'Itinerary.pickle', "rb" ) )
#results, x_star, y_star,status = loaded_results[0],loaded_results[1],loaded_results[2],loaded_results[3]

### 9. Print Model Results

In [17]:
# Printing decision variable results in a table form
np.set_printoptions(threshold = sys.maxsize)
df_res = pd.DataFrame(index = data['name'].values.tolist())

for i in range(len(x_star)):
    colname = name[i]
    df_res[colname] = x_star[i,:].tolist()
        
df_res = df_res.astype(int)
df_res.transpose()
# df_res.to_excel("Results.xlsx")

,Marina Bay Sands,Park Avenue Changi Hotel,Hard Rock Hotel,Shangri La Hotel,Genting Hotel Jurong,Ibis Styles on MacPherson,Universal Studios Singapore,Singapore Art Museum,Gardens By The Bay,Singapore Zoo,Singapore DUCKtours,ION Orchard,Botanic Gardens,Istana,Fort Siloso,Fort Canning,Clarke Quay,Merlion,Esplanade,Singapore Flyer,Jurong Lake Gardens,East Coast Park,Jewel,Singapore Discovery Centre,Singapore Science Centre,National Gallery,Chinatown,Keong Saik Rd,Kampong Lorong Buangkok,Haw Par Villa,Little India,Arab Street
Marina Bay Sands,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0
Park Avenue Changi Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Hard Rock Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Shangri La Hotel,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Genting Hotel Jurong,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Ibis Styles on MacPherson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Universal Studios Singapore,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
Singapore Art Museum,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
Gardens By The Bay,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Singapore Zoo,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
# Printing itinerary results in a human readable form
for z in range(accomodations_count):
    if x_star[z,:].sum() == days:
        startingnodesj = [i for i in range(len(x_star[z,:])) if x_star[z,i] == 1]
        hotelchosen = z

trip_dict = {}

for s, activityindex in enumerate(startingnodesj):
    day = s + 1
    activityno = activityindex
    day_list = [hotelchosen, activityindex]
    while activityno != hotelchosen:
        activityno = x_star[activityno,:].tolist().index(1)
        day_list.append(activityno)
    trip_dict[f"Day {day}"] = day_list

dateofday = startdate
day = 0

print(f"This is your {city} itinerary from {startdate} to {enddate}")
print("")

for each in trip_dict:
    prevact = hotelchosen
    for n, act in enumerate(trip_dict[each]):
        endtimeadd = travelact_mat[prevact,act]
        starttimeadd = endtimeadd - estduration[act]
        if prevact == hotelchosen:
            actstart = (starttimeofday + datetime.timedelta(hours = starttimeadd))
            actend = (starttimeofday + datetime.timedelta(hours = endtimeadd))
        else:
            actstart = (prevactstart + datetime.timedelta(hours = starttimeadd))
            actend = (prevactstart + datetime.timedelta(hours = endtimeadd))
        if act == hotelchosen and n == 0:
            print("Start of ", dateofday.strftime("%m/%d/%Y"), " from ", name[act], starttimeofday)
        elif act == hotelchosen and n != 0:
            print("End of ", dateofday.strftime("%m/%d/%Y"), " at ", name[act])
            print("Weather Forecast: ")
            for x in forecast_results[day]:
                print(x)
            print("\n")
        else:
            print(each, (datetime.datetime.min + actstart).strftime('%I:%M %p').lstrip('0'), " - ", (datetime.datetime.min + actend).strftime('%I:%M %p').lstrip('0'), ":", name[act])
        prevact = act
        prevactstart = actend
    day += 1
    dateofday += datetime.timedelta(days = 1)

This is your Singapore itinerary from 2023-04-20 to 2023-04-24

Start of  04/20/2023  from  Marina Bay Sands 9:00:00
Day 1 9:15 AM  -  11:15 AM : Fort Siloso
Day 1 11:19 AM  -  7:19 PM : Universal Studios Singapore
Day 1 7:31 PM  -  9:31 PM : National Gallery
End of  04/20/2023  at  Marina Bay Sands
Weather Forecast: 
From 0900hrs to 1100hrs: Light Rain
From 1100hrs to 1200hrs: Heavy Rain
From 1200hrs to 1400hrs: Light Rain
From 1400hrs to 1500hrs: Moderate Rain
From 1400hrs to 2200hrs: No Rain


Start of  04/21/2023  from  Marina Bay Sands 9:00:00
Day 2 9:05 AM  -  11:05 AM : Clarke Quay
Day 2 11:05 AM  -  12:05 PM : Fort Canning
Day 2 12:08 PM  -  3:08 PM : Singapore Art Museum
Day 2 3:10 PM  -  5:10 PM : Little India
Day 2 5:11 PM  -  7:11 PM : Arab Street
Day 2 7:15 PM  -  9:15 PM : Singapore DUCKtours
End of  04/21/2023  at  Marina Bay Sands
Weather Forecast: 
From 0900hrs to 1000hrs: Light Rain
From 1000hrs to 1100hrs: Heavy Rain
From 1100hrs to 1500hrs: Light Rain
From 1500hrs t

In [19]:
# Visualising routes on the Singapore map
flattrip = []
for day in trip_dict:
    flattrip.extend(trip_dict[day])

colorlist = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']
SGmap1 = folium.Map(location=[lat[2], lon[2]], tiles="Stamen Terrain", zoom_start=11) #Stamen Terrain, OpenStreetMap
d = 0
route_groups = []
client = ors.Client(key = '5b3ce3597851110001cf624835774ebd2fcc4d7baf8b08f85b315072')

for day, activities in trip_dict.items():
    day_routes = []
    for i in range(len(activities) - 1):
        coordinates = [[lon[activities[i]], lat[activities[i]]], [lon[activities[i+1]], lat[activities[i+1]]]]
        route = client.directions(coordinates=coordinates, profile='driving-car', format='geojson')
        day_routes.append(route)
    route_group = folium.FeatureGroup(name=f"Day {d+1} Route")
    for route in day_routes:
        gj = folium.GeoJson(
            name="route",
            data={
                "type": "FeatureCollection",
                "features": [
                    {
                        "type": "Feature",
                        "geometry": route["features"][0]["geometry"],
                    }
                ],
            },
            style_function=lambda feature, d=d: {"color": colorlist[d % len(colorlist)]},
        )
        gj.add_to(route_group)
        # Add hotel and activity markers for this day's route
        for i in range(len(activities)):
            marker_color = "blue" if activities[i] < accomodations_count else "red"
            marker_label = "H" if activities[i] < accomodations_count else str(i)
            folium.Marker(
                location=[lat[activities[i]], lon[activities[i]]],
                icon=BeautifyIcon(
                    icon_shape="marker",
                    number=marker_label,
                    spin=True,
                    text_color=marker_color,
                    background_color="#FFF",
                    inner_icon_style="font-size:12px;padding-top:-5px;",
                ),).add_to(route_group)
    route_groups.append(route_group)
    d += 1

# Add route groups to map and layer control
for route_group in route_groups:
    route_group.add_to(SGmap1)
folium.LayerControl().add_to(SGmap1)

SGmap1